In [2]:
import pandas as pd
from myopic_mces import MCES, construct_graph
from myopic_mces.MCES_ILP import get_ilp_itself
from itertools import product
from tqdm.cli import tqdm
from IPython.display import clear_output
from functools import lru_cache
from myopic_mces import construct_graph
import pulp
import networkx as nx

smiles_a, smiles_b = ['CC(=O)OC1=CC=CC=C1C(=O)O'], ['CN1C=NC2=C1C(=O)N(C(=O)N2C)C']
# smiles_a, smiles_b = ['CCc1ccccc1'], ['C=Cc1ccccc1']
G1 = construct_graph(smiles_a[0])
G2 = construct_graph(smiles_b[0])
ilp = get_ilp_itself(G1, G2, -1)

In [ ]:
# c = [ mappable_edge_pairs * weight + edges_not_mapped_penalty_bond ]
# s.t. 
# One node can map to only one other node, both way, assignment problem ...
# The mapping of the edges has to match the mapping of the nodes:
#   yik is node i mapped to k
#   yil is node i mapped to l
#   then sum cikl <= yil + yik #
# # cikl is 0. 0 <= yil + yik # disabled
# cikl is 1
# Every edge must be mapped, or not not mapping has to be 1

In [ ]:
# Rows and columns

# Rows of edges
# Columns of edges
# Can I answer: Is this solvable within C<1, quickly?

# Each edge to other edge, or edge penalty
# 

In [2]:
smiles_all = pd.read_csv('MassSpecGym.tsv', sep='\t').smiles.unique()
smiles_a = smiles_all[:100]
smiles_b = smiles_all[100:200]

In [3]:
ilp.solve()
ilp.objective.value()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/tornikeo/micromamba/envs/pb/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/56b4833c15af4ae9b805627f2a09aedc-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/56b4833c15af4ae9b805627f2a09aedc-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 437 COLUMNS
At line 1427 RHS
At line 1860 BOUNDS
At line 1995 ENDATA
Problem MODEL has 432 rows, 134 columns and 683 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 27 - 0.00 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 6 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 6 strengthened rows, 0 substitutions
Cgl0004I processed model has 92 rows, 81 columns (81 integer (81 of which binary)) and 327 elements
Cutoff increment increased from 1e-05 to 0.4999
Cbc0038I Initial state - 0 integers

27.0

In [4]:

from scipy.optimize import milp, LinearConstraint, Bounds
import numpy as np

import numpy as np
import pulp

def extract_lp_matrices(model):
    """
    Extracts (c, A, ub, lb) matrices from a PuLP ILP model where all variables are binary.
    """
    # Ensure the problem is a maximization problem

    # Get list of variables in sorted order
    variables = list(model.variables())
    variables.sort(key=lambda v: v.name)
    var_hash = [v.hash for v in variables]
    num_vars = len(variables)

    # Extract objective coefficients (c)
    c = np.array([model.objective.get(v, 0) for v in variables])
    if model.sense == pulp.LpMaximize:
        c = -c
        
    # Extract constraint matrix (A) and bounds (ub)
    constraint_list = list(model.constraints.values())
    num_constraints = len(constraint_list)

    A = np.zeros((num_constraints, num_vars))
    ub = np.zeros(num_constraints)

    for i, constraint in enumerate(constraint_list):
        ub[i] = -constraint.constant
        for var, coeff in constraint.items():
            j = var_hash.index(var.hash)  # Find index of variable
            A[i, j] = coeff
        # Convert equality constraints to two inequalities if needed
        if constraint.sense == pulp.LpConstraintEQ:
            A = np.vstack([A, -A[i, :]])
            ub = np.hstack([ub, -ub[i]])
    # All variables are binary, so lower and upper bounds are known
    return c, A, ub


for i,(a,b) in enumerate(zip(smiles_a, smiles_b)):
    G1 = construct_graph(a)
    G2 = construct_graph(b)
    ilp = get_ilp_itself(G1, G2, -1)
    # c, A, ub = extract_lp_matrices(ilp)

    # res = milp(
    #     c=c, 
    #     constraints=LinearConstraint(
    #         A=A,
    #         ub=ub,
    #     ), 
    #     integrality=np.ones_like(c), 
    #     bounds=Bounds(0, 1)
    # )
    # ilp.solve()
    # print(res.fun, ilp.objective.value())
    ilp.writeLP(f'mps/{i:03}.lp')

KeyboardInterrupt: 

In [ ]:
smiles_a.tolist()

['CC(=O)N[C@@H](CC1=CC=CC=C1)C2=CC(=CC(=O)O2)OC',
 'C[C@H]1CCCC(=O)CCC/C=C/C2=C(C(=CC(=C2)O)O)C(=O)O1',
 'CC1CCCC(=O)CCCC=CC2=C(C(=CC(=C2)O)O)C(=O)O1',
 'CC(C)(C=C)C1(C2=CC=CC=C2N(C1=O)O)CC3C(=O)N/C(=C/C4=CN=CN4)/C(=O)N3',
 'C[C@@H]1C[C@H]2[C@H](O2)/C=C\\C(=O)CC(=O)O1',
 'CC12CC3=C(C(O1)C[C@@H](O2)C(=O)O)C(=C4C(=O)C=C(C(=O)C4=C3O)OC)O',
 'C[C@H]1CN([C@@H]2CC3=CNC4=CC=CC(=C34)[C@H]2[C@@H]1OC(=O)C)C',
 'CC[C@H](C)[C@H]1C(=O)O[C@@H](C(=O)N([C@H](C(=O)O[C@@H](C(=O)N([C@H](C(=O)O[C@@H](C(=O)N1C)C(C)C)C(C)C)C)C(C)C)C(C)C)C)C(C)C',
 'CCC(C)C1C(=O)OC(C(=O)N(C(C(=O)OC(C(=O)N(C(C(=O)OC(C(=O)N1C)C(C)C)C(C)C)C)C(C)C)C(C)C)C)C(C)C',
 'CC1=C[C@@H]2[C@]([C@@H](C1=O)O)([C@]3([C@@H]([C@H]([C@H]([C@@]34CO4)O2)O)OC(=O)C)C)CO',
 'CC1=CC2C(C(C1=O)O)(C3(C(C(C(C34CO4)O2)O)OC(=O)C)C)CO',
 'CC1=CC(=O)C2=C(C3=C(C=C(C=C3C=C2O1)OC)O)O',
 'CC1=C[C@@H]2[C@@]3(CC1)COC(=O)/C=C(/CCO[C@@H](/C=C/C=C\\C(=O)O[C@H]4[C@]3([C@]5(CO5)[C@@H](C4)O2)C)[C@H](C)O)\\C',
 'CC(C)C[C@H]1C(=O)O[C@@H](C(=O)N([C@H](C(=O)O[C@@H](C(=O)N([C

In [ ]:
ilp.solve()
ilp.objective.value()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/tornikeo/micromamba/envs/pb/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/8127686345bf46d09fabdea4974104d7-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/8127686345bf46d09fabdea4974104d7-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 437 COLUMNS
At line 1427 RHS
At line 1860 BOUNDS
At line 1995 ENDATA
Problem MODEL has 432 rows, 134 columns and 683 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 27 - 0.00 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 6 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 6 strengthened rows, 0 substitutions
Cgl0004I processed model has 92 rows, 81 columns (81 integer (81 of which binary)) and 327 elements
Cutoff increment increased from 1e-05 to 0.4999
Cbc0038I Initial state - 0 integers

27.0

In [ ]:
A.shape, c.shape

((460, 134), (134,))

In [ ]:
# import numpy as np

# # Set random seed for reproducibility
# np.random.seed(42)

# # Problem size (small example)
# num_vars = 5  # Number of binary decision variables
# num_constraints = 3  # Number of constraints

# # Generate random cost vector c
# c = -np.random.randint(1, 10, size=num_vars)

# # Generate random constraint matrix A
# A = np.random.randint(0, 5, size=(num_constraints, num_vars))

# # Generate random upper bounds for constraints
# ub = np.random.randint(5, 15, size=num_constraints)

# # Lower bounds (all zero for inequalities)
# lb = np.zeros(num_constraints)

# # Define constraints
# constraints = LinearConstraint(A, lb, ub)

# # Define integer constraints (all variables are binary)
# integrality = np.ones(num_vars)

# # Solve using scipy.optimize.milp
# res = milp(c=c, constraints=constraints, integrality=integrality, bounds=(0, 1))

# # Display results
# res.x, res.fun

In [ ]:
c

array([0.5, 0.5, 1. , 0. , 0. , 2. , 1. , 1. , 1. , 0. , 0. , 2. , 1. ,
       1. , 1. , 1. , 1. , 1. , 0. , 0. , 1.5, 0. , 0. , 1.5, 0. , 0. ,
       1.5, 0. , 0. , 1.5, 0. , 0. , 1.5, 0. , 0. , 1.5, 0.5, 0.5, 1. ,
       1. , 1.5, 1.5, 1. , 1.5, 1.5, 1.5, 1.5, 1.5, 2. , 1.5, 1. , 1.5,
       2. , 1.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. ])

In [ ]:
from itertools import product



def binary_milp(c, A, ub):
    num_vars = len(c)
    best_x = None
    best_obj = float('inf')

    # Generate all possible binary vectors
    for x in product([0, 1], repeat=num_vars):
        x = np.array(x)
        # Check feasibility: lb <= A @ x <= ub
        if np.all(A @ x <= ub):
            obj_value = c @ x  # Compute objective function

            # Update best solution if this one is better
            if obj_value < best_obj:
                best_obj = obj_value
                best_x = x.copy()
                print(best_obj)

    return best_x, best_obj

# Solve using our simple binary_milp function
binary_milp_solution = binary_milp(c, A, ub)
binary_milp_solution

KeyboardInterrupt: 

In [ ]:
from itertools import product
from scipy.optimize import linprog

def greedy_feasible_solution(c, A, ub, lb):
    """Find a quick feasible solution to establish an upper bound."""
    num_vars = len(c)
    x = np.zeros(num_vars)  # Start with all zeros

    # Sort variables by cost-effectiveness (lowest c_i first, since we're minimizing)
    indices = np.argsort(c)

    # Try flipping variables to 1 while maintaining feasibility
    for i in indices:
        x[i] = 1
        if np.any(A @ x > ub) or np.any(A @ x < lb):
            x[i] = 0  # Revert if infeasible

    return x, c @ x  # Return feasible solution and its objective value


def lp_relaxation(c, A, ub, lb):
    """Solve LP relaxation to get fractional values."""
    res = linprog(c, A_ub=A, b_ub=ub, bounds=(0, 1), method='highs')
    return res.x if res.success else None, res.fun if res.success else float('inf')


def branch_and_bound(c, A, ub, lb, x_partial, depth, best_x, best_obj):
    """Recursive branch-and-bound solver."""
    num_vars = len(c)

    # Solve LP relaxation
    relaxed_x, relaxed_obj = lp_relaxation(c, A, ub, lb)

    # Prune if relaxation is worse than best known solution
    if relaxed_obj >= best_obj:
        return best_x, best_obj

    # Check if the relaxed solution is already integer
    if np.all(np.isclose(relaxed_x, np.round(relaxed_x))):
        return relaxed_x, relaxed_obj

    # Select the most fractional variable for branching
    fractional_vars = np.abs(relaxed_x - 0.5)
    branch_var = np.argmax(fractional_vars)

    # Try setting the variable to 0 and 1 recursively
    for val in [0, 1]:
        new_x_partial = x_partial.copy()
        new_x_partial[branch_var] = val

        # Check feasibility
        if np.all(A @ new_x_partial <= ub) and np.all(A @ new_x_partial >= lb):
            obj_value = c @ new_x_partial
            if obj_value < best_obj:
                best_x, best_obj = new_x_partial.copy(), obj_value

            # Recursively branch
            best_x, best_obj = branch_and_bound(c, A, ub, lb, new_x_partial, depth + 1, best_x, best_obj)

    return best_x, best_obj


def binary_milp(c, A, ub, lb):
    """Main function to solve binary integer programming using branch-and-bound."""
    num_vars = len(c)

    # Get initial feasible solution
    best_x, best_obj = greedy_feasible_solution(c, A, ub, lb)

    # Start branch-and-bound
    x_partial = np.full(num_vars, -1)  # -1 means unassigned
    best_x, best_obj = branch_and_bound(c, A, ub, lb, x_partial, 0, best_x, best_obj)

    return best_x, best_obj

# Solve using the improved binary_milp function
binary_milp_solution = binary_milp(-c, A, ub, np.full_like(ub, -np.inf))
binary_milp_solution

(array([-0., -0.,  1.,  0.,  0.,  1., -0., -0.,  1.,  0.,  0.,  1., -0.,
         0.,  1., -0., -0.,  1.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,
         1.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,  1., -0.,  0.,  1.,
         1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.,  0.,  0.,  0.,  0.,  0., -0.,  0.,  0.,  0.,  0.,  0.,
         0., -0., -0., -0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0., -0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0., -0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.]),
 -40.0)

In [ ]:
np.sort(c)[::-1]

array([2. , 2. , 2. , 2. , 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5,
       1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1. , 1. , 1. , 1. , 1. , 1. ,
       1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 0.5, 0.5, 0.5, 0.5, 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. ])